In [3]:
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import glob
reload(ld)

<module 'sib_ldsc_z' from 'c:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

# Defining the PDF and the Log Likelihoods

The likelihood for a SNP $i$ is:

$$
l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) - \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i
$$

And its derivative:

$$
\frac{dl}{dV} = r_i S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}
$$

In [4]:
np.random.seed(123)

# N = int(100)
# S_size = int(N/2)
# S = np.array([np.array([[.5, 0], [0, .8]]),
#     np.array([[0.5, 0], [0, 0.8]])] * S_size )/N
# V = np.identity(2) * 0.5

N = int(1e4)
S = np.array([[[1e-4, -5 * 1e-5], [-5 * 1e-5, 1e-4]]] * N)
V = np.array([[0.5, 0.25], [0.25, 0.5]])


# N = int(1e4)
# S = np.array([[[1e-4, -5 * 1e-5, -4 * 1e-5], [-5 * 1e-5, 1e-4, -4 * 1e-5], [-5 * 1e-5, -4 * 1e-5, 1e-4]]] * N)
# V = np.array([[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5]])

model = ld.sibreg(S = S)
model.simdata(V/N, N, simld = True)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for LD Scores given. Generating a vector of ones for l
Simulated LD scores!


In [6]:
# Testing derivatives
aderiv = ld._grad_ll_v(ld.Vmat2V(V, N), model.z[0, :], model.S[0],  model.l[0], N)

nderiv = ld._num_grad_V(ld.Vmat2V(V, N), model.z[0, :], model.S[0], model.l[0],  N)

np.allclose(aderiv, nderiv)

True

In [7]:
# solving
output, result = model.solve()
print(result)

No initial guess provided.
Making zero vector
Logll : 52566.39739772634
V : [1.e-06 1.e-06 0.e+00]
Logll : 37705.461467003784
V : [7.15044642e-01 6.99080817e-01 1.32215374e-06]
Logll : 37684.34116340675
V : [0.71066104 0.69418528 0.01163596]
Logll : 37603.957076848135
V : [0.69312664 0.67460312 0.05817451]
Logll : 37348.13221022577
V : [0.62298902 0.59627449 0.24432871]
Logll : 37211.22443983857
V : [0.46385218 0.44014044 0.58653786]
Logll : 37190.06911605655
V : [0.51699619 0.49559746 0.50859603]
Logll : 37189.29547683617
V : [0.51551085 0.4905291  0.52288458]
Logll : 37189.069860080555
V : [0.51585899 0.49213738 0.53214122]
Logll : 37189.053471743624
V : [0.51808524 0.49210471 0.53244743]
Logll : 37189.052377922715
V : [0.51800691 0.49256668 0.53223161]
Logll : 37189.05234751952
V : [0.51801913 0.49265042 0.53215488]
      fun: 37189.05234751952
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00473645, -0.00023539,  0.00279632])
  message: b'CONVERGENCE: 

In [8]:
output

{'v1': 0.518019126543805, 'v2': 0.49265041748579746, 'r': 0.5321548832103554}

In [11]:
print("Real Parameters:")
print(f"v1 = {V[0, 0]}")
print(f"v2 = {V[1, 1]}")
print(f"r = {V[0, 1]/np.sqrt(V[0, 0] * V[1, 1])}")

Real Parameters:
v1 = 0.5
v2 = 0.5
r = 0.5
